In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
X = torch.rand(2,20)
print(X)
print(net(X))

tensor([[0.4696, 0.8409, 0.2254, 0.1035, 0.5493, 0.6544, 0.7111, 0.5228, 0.7014,
         0.9816, 0.6352, 0.2079, 0.5762, 0.7195, 0.5036, 0.6768, 0.2030, 0.1389,
         0.2376, 0.2197],
        [0.1633, 0.7260, 0.3449, 0.1628, 0.7112, 0.4431, 0.3441, 0.2542, 0.3543,
         0.9336, 0.7441, 0.5434, 0.8087, 0.8029, 0.7299, 0.5552, 0.8546, 0.1124,
         0.7419, 0.1730]])
tensor([[ 0.0716, -0.2320, -0.1550, -0.1868,  0.0307, -0.1387,  0.3237,  0.0457,
         -0.0634,  0.4934],
        [ 0.1279, -0.1973, -0.1493, -0.1213,  0.1058, -0.1211,  0.2945,  0.0603,
         -0.0385,  0.4337]], grad_fn=<AddmmBackward0>)


### 每个块必须提供的基本功能：
- 将输入数据作为其前向传播函数的参数。

- 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。

- 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。

- 存储和访问前向传播计算所需的参数。

- 根据需要初始化模型参数。

In [7]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接层
    def __init__(self):
        # 调用MLP的父类 Module的构造函数来执行必要的初始化
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层
        
    # 定义模型的前向传播，即如何根据输入X返回所需的模型参数
    def forward(self,X):
        # 注意 这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))
    #除非我们实现一个新的运算符， 否则我们不必担心反向
    #传播函数或参数初始化， 系统将自动生成这些。

块的一个主要优点是它的多功能性。 我们可以子类化块以创建层（如全连接层的类）、 整个模型（如上面的MLP类）或具有中等复杂度的各种组件。 我们在接下来的章节中充分利用了这种多功能性， 比如在处理卷积神经网络时。

In [8]:
net = MLP()
net(X)

tensor([[-0.0206, -0.1011,  0.1789,  0.0637, -0.0175,  0.0306,  0.3341, -0.1242,
          0.1708,  0.2154],
        [-0.0039, -0.1338,  0.2872,  0.1136,  0.0691, -0.0273,  0.3492, -0.1579,
          0.1773,  0.1971]], grad_fn=<AddmmBackward0>)

### 为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：

- 一种将块逐个追加到列表中的函数。

- 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的MySequential类提供了与默认Sequential类相同的功能。

In [13]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [14]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.1946,  0.1638, -0.1115, -0.0632, -0.1620, -0.1213,  0.0385, -0.0238,
          0.2288, -0.0128],
        [ 0.1930,  0.2300, -0.0585, -0.0695, -0.2508, -0.1058, -0.0435,  0.0132,
          0.1238, -0.0474]], grad_fn=<AddmmBackward0>)

### 们可能希望在前向传播函数中执行Python的控制流。 此外，我们可能希望执行任意的数学运算， 而不是简单地依赖预定义的神经网络层。

In [15]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)
    
    def forward(self,X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 使用复全连接层 这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [16]:
net = FixedHiddenMLP()
net(X)

tensor(0.0478, grad_fn=<SumBackward0>)

### 我们可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块

In [18]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1394, grad_fn=<SumBackward0>)

### 小结
- 一个块可以由许多层组成；一个块可以由许多块组成。

- 块可以包含代码。

- 块负责大量的内部处理，包括参数初始化和反向传播。

- 层和块的顺序连接由Sequential块处理。